## 라이브러리 준비

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By

import pandas as pd
import time
from tqdm import tqdm

## 웹사이트 스크랩

### Team Statistics

In [ ]:
try:
    # WebDriver 설정
    driver = webdriver.Chrome()

    # 웹 페이지 열기
    driver.get("https://1xbet.whoscored.com/Regions/252/Tournaments/2/Seasons/9618/Stages/22076/TeamStatistics/England-Premier-League-2023-2024")
    time.sleep(1)   # 광고 뜨면 닫아주기

    # 탭 목록 확인 (Summary, Defensive, Offensive, xG)
    tabs = driver.find_elements(By.XPATH, "//ul[@id='stage-team-stats-options']/li/a")
    for tab in tabs:
        if tab.text in ["Summary", "Defensive", "Offensive", "xG"]: # 필요한 tab 데이터 수집
            print(tab.text)
            tab.click()
            time.sleep(2)

            # 테이블 헤더 수집
            thead = driver.find_element(By.XPATH, "//div[@class='statistics-table-tab' and not(@style='display: none;')]//thead")
            headers = [header.text for header in thead.find_elements(By.TAG_NAME, "th")]

            # 테이블 데이터 수집
            rows = []
            tbody = driver.find_element(By.XPATH, "//div[@class='statistics-table-tab' and not(@style='display: none;')]//tbody")
            trs = tbody.find_elements(By.TAG_NAME, "tr")

            for tr in trs:
                row = [td.text for td in tr.find_elements(By.TAG_NAME, "td")]

                # 테이블 데이터 전처리
                row[0] = row[0].split('. ')[1] # 팀 이름 앞의 '숫자.' 제거
                rows.append(row)

            # Pandas DataFrame 생성 및 데이터 추가
            df = pd.DataFrame(rows, columns=headers)

            # DataFrame을 CSV로 저장
            filename = f"./data/TeamStatistics_{tab.text}.csv"
            df.to_csv(filename, index=False, encoding="utf-8")
            print(filename, "\n")

finally:
    # 드라이버 종료
    driver.quit()


Summary
./data/TeamStatistics_Summary.csv

Defensive
./data/TeamStatistics_Defensive.csv

Offensive
./data/TeamStatistics_Offensive.csv

xG
./data/TeamStatistics_xG.csv



### Player Statistics

In [ ]:
try:
    # WebDriver 설정
    driver = webdriver.Chrome()

    # 웹 페이지 열기
    driver.get("https://1xbet.whoscored.com/Regions/252/Tournaments/2/Seasons/9618/Stages/22076/PlayerStatistics/England-Premier-League-2023-2024")
    time.sleep(1)   # 광고 뜨면 닫아주기

    # 탭 목록 확인
    tabs = driver.find_elements(By.XPATH, "//ul[@id='stage-top-player-stats-options']/li/a")
    for tab in tabs:
        if tab.text in ["Summary", "Defensive", "Offensive", "Passing", "xG"]:  # 필요한 tab 데이터 수집
            print(tab.text)
            tab.click()
            time.sleep(2)

            # 전체 페이지 수 확인
            total_pages_elem   = driver.find_element(By.XPATH, "//div[@class='statistics-table-tab' and not(@style='display: none;')]//*[@id='totalPages']")
            total_pages = int(total_pages_elem.get_attribute("value"))
            print("Total pages: ", total_pages)

            # 테이블 헤더 수집
            thead = driver.find_element(By.XPATH, "//div[@class='statistics-table-tab' and not(@style='display: none;')]//thead")
            headers = [header.text for header in thead.find_elements(By.TAG_NAME, "th")]

            # 테이블 헤더 전처리
            headers = [x for x in headers if x] # 빈 문자열 제거
            headers.insert(0, 'Rank')           # headers[0]에 'Rank' 삽입
            headers.insert(2, 'Team')           # headers[2]에 'Team' 삽입
            headers.insert(3, 'Age')            # headers[3]에 'Age' 삽입
            headers.insert(4, 'Position')       # headers[4]에 'Position' 삽입

            # 테이블 데이터 수집
            rows = []
            for page in tqdm(range(total_pages)):
                tbody = driver.find_element(By.XPATH, "//div[@class='statistics-table-tab' and not(@style='display: none;')]//tbody")
                trs = tbody.find_elements(By.TAG_NAME, "tr")

                for tr in trs:
                    row = [td.text for td in tr.find_elements(By.TAG_NAME, "td")]

                    # 테이블 데이터 전처리
                    info = row[0].split('\n') # 줄 바꿈 문자로 분리
                    rank = info[0] # 'Rank'
                    name = info[1] # 'Player'
                    team = info[2].split(', ')[0] # 'Team'
                    age = info[2].split(', ')[1] # 'Age'
                    position = ','.join(row[0].split('\n')[2].split(', ')[2:]) # Position
                    row = [rank, name, team, age, position] + [x for x in row[2:] if x] # 빈 문자열 제거
                    rows.append(row)
                
                # 다음 페이지로 이동
                next_button = driver.find_element(By.XPATH, "//div[@class='statistics-table-tab' and not(contains(@style, 'display: none'))]//a[@id='next']")
                next_button.click()
                time.sleep(1)

            # Pandas DataFrame 생성 및 데이터 추가
            df = pd.DataFrame(rows, columns=headers)

            # DataFrame을 CSV로 저장
            filename = f"./data/PlayerStatistics_{tab.text}.csv"
            df.to_csv(f"./data/PlayerStatistics_{tab.text}.csv", index=False, encoding="utf-8")
            print(filename, "\n")

finally:
    # 드라이버 종료
    driver.quit()


Summary
Total pages:  32


100%|██████████| 32/32 [01:03<00:00,  2.00s/it]


./data/PlayerStatistics_Summary.csv

Defensive
Total pages:  32


100%|██████████| 32/32 [01:04<00:00,  2.01s/it]


./data/PlayerStatistics_Defensive.csv

Offensive
Total pages:  32


...

./data/PlayerStatistics_Offensive.csv

Passing
Total pages:  32


100%|██████████| 32/32 [01:02<00:00,  1.96s/it]


./data/PlayerStatistics_Passing.csv

xG
Total pages:  30


100%|██████████| 30/30 [00:56<00:00,  1.87s/it]


./data/PlayerStatistics_xG.csv



## 데이터 병합

### Team Statistics

In [ ]:
# df 모두 불러오기
dfs = [pd.read_csv(f"./data/TeamStatistics_{x}.csv") for x in ["Summary", "Defensive", "Offensive", "xG"] ]

# 전처리 ⚠️
# Summary, Defensive, Offensive에 "Shots pg" 열이 모두 포함되어 있으나
# Summary, Offensive의 "Shots pg"은 경기당 시도한 슈팅 수
# Defensive의 "Shots pg"은 경기당 허용한 슈팅 수
dfs[1] = dfs[1].rename(columns={'Shots pg': 'Shots Against pg'})  # Defensive의 "Shots pg"은 경기당 허용한 슈팅 수

# 공통 컬럼 (데이터프레임 병합의 기준)
merge_keys = ['Team']

# 0번(Summary) df에 나머지 df 병합
df_merged = dfs[0]

for i in range(1, len(dfs)):
    # 기존 데이터프레임에 없는 새로운 컬럼만 찾기 (set 차집합 연산 활용)
    new_columns = list(set(dfs[i].columns) - set(df_merged.columns))
    # 병합할 때 merge_keys + 새로운 컬럼만 선택
    df_merged = df_merged.merge(dfs[i][merge_keys + new_columns], on=merge_keys, how="left")

filename = "./data/TeamStatistics.csv"
df_merged.to_csv("./data/TeamStatistics.csv", index=False, encoding="utf-8")
print(filename)
df_merged.head()


./data/TeamStatistics.csv


,Team,Goals,Shots pg,Discipline,Possession%,Pass%,AerialsWon,Rating,Fouls pg,Offsides pg,...,Interceptions pg,Shots pg(Defensive),Dribbles pg,Shots OT pg,Fouled pg,Shots,xG/Shots,Goals*,xG,xGDiff
0,Manchester City,96,18.2,522,65.4,90.5,8.7,6.97,7.6,1.1,...,6.1,7.7,9.9,7.3,10.8,693,0.13,94,87.07,6.93
1,Arsenal,91,17.3,622,58.4,86.8,13.6,6.87,10.3,2.3,...,7.7,8.5,7.4,6.0,10.4,657,0.13,86,84.51,1.49
2,Liverpool,86,20.8,655,61.3,85.8,15.6,6.86,12.2,2.6,...,7.7,10.9,8.7,7.3,9.9,790,0.12,80,93.30,-13.30
3,Newcastle,85,14.5,771,52.2,82.7,12.0,6.78,10.2,1.6,...,7.5,14.1,9.3,5.6,12.3,551,0.15,83,83.98,-0.98
4,Chelsea,77,14.4,1054,59.0,87.5,11.8,6.74,11.7,2.2,...,7.8,14.6,10.5,5.7,11.9,549,0.15,76,80.18,-4.18


### Player Statistics

In [ ]:
# df 모두 불러오기
dfs = [pd.read_csv(f"./data/PlayerStatistics_{x}.csv") for x in ["Summary", "Defensive", "Offensive", "Passing", "xG"] ]

# 공통 컬럼 (데이터프레임 병합의 기준)
merge_keys = ['Player', 'Team', 'Age', 'Position', 'Apps', 'Mins']

# 0번(Summary) df에 나머지 df 병합
df_merged = dfs[0]

for i in range(1, len(dfs)):
    # 새로운 컬럼만 찾기 (set 차집합 연산 활용)
    new_columns = list(set(dfs[i].columns) - set(df_merged.columns))
    # 병합할 때 merge_keys + 새로운 컬럼만 선택
    df_merged = df_merged.merge(dfs[i][merge_keys + new_columns], on=merge_keys, how="left")

filename = "./data/PlayerStatistics.csv"
df_merged.to_csv(filename, index=False, encoding="utf-8")
print(filename)
df_merged.head()

./data/PlayerStatistics.csv


,Rank,Player,Team,Age,Position,Apps,Mins,Goals,Assists,Yel,...,Disp,Crosses,AvgP,LongB,ThrB,Shots,xG/Shots,xG,xGDiff,xG/90
0,1,Bukayo Saka,Arsenal,23,"D(L),M(CLR)",35,2937,16,9,4,...,1.5,1.3,37.4,0.8,-,108.0,0.15,16.23,-0.23,0.5
1,2,Rodri,Man City,28,"D(C),DMC",34,2937,8,9,8,...,0.9,-,106.9,7.1,0.2,62.0,0.07,4.46,3.54,0.14
2,3,Michael Olise,Crystal Palace,23,AM(CLR),14(5),1277,10,6,-,...,1.7,1.3,32.1,1.3,0.3,57.0,0.11,6.22,3.78,0.44
3,4,Phil Foden,Man City,24,"AM(CLR),FW",33(2),2869,19,8,2,...,1.4,0.5,47.7,1.2,0.3,105.0,0.11,11.28,7.72,0.35
4,5,Cole Palmer,Chelsea,22,"AM(CR),FW",29(4),2618,22,11,7,...,1.3,0.5,39.5,2.1,0.4,109.0,0.16,17.85,4.15,0.61
